#### IMPORTS

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import gamma
import time
import math
import matplotlib as mp
import scipy as sp
import pylab as py
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.animation

from ipynb.fs.full.Neutron_Star_SPH_sph import *
from ipynb.fs.full.Neutron_Star_SPH_eos import *

# **__Neutron Star Object Class & Initialization__**


### **__NEUTRON STAR OBJECT ATTRIBUTES:__**
- tag
- mass (SMs)
- radius (km)
- m (SPH point mass)
- num_points
- eos
- lagrange_tag
- kernel_type
- eta
- nu

### **__POINT ATTRIBUTES:__**

- position (N x 3)
- vel (N x 3)
- acc (N x 3)
- density (N x 1)
- pressure (N x 1)

## **__Get Acceleration__**

In [22]:
def self_gravity_radius(dx, dy, dz): 
    """ dx, dy, dz  are (M x N) matrices of separations. 
    Returns (N x 3) array 
    Each row of the return array represents r_theorietical, the average distance of all other points b to the point in that row (a), which is the theoretical center of mass of  gravitational object..
    """
    N = np.shape(dx)[0]
    dx_theoretical = dx.sum(axis=0)
    dy_theoretical = dy.sum(axis=0)
    dz_theoretical = dz.sum(axis=0)
    return dx_theoretical/(N-1),dy_theoretical/(N-1),dz_theoretical/(N-1)

def getAcc(star):
    """  Calculate the acceleration on each SPH particle given current positions and velocities.
    'pos' contains particle coordinates (N x 3 matrix), 'vel' contains particle velocities (N x 3 matrix), returns 'a' containing accelerations (another N x 3 matrix)    """ 
    m = star.m # m = particle mass
    pos = star.pos # point positions relative to COM
    vel = star.vel # point velocities
    rho = star.rho # star density
    kernel_type_ = star.kernel_type # Wendland or Gaussian kernel
    
    # ======================= 1. Pressure from density, EOS ======================= #
    eos = star.eos
    P = eos.P(star.rho) # Pressure as a function of density
    clmbda = star.clmbda # constant for potential energy
    
    # ======================= 2. SPH Update ======================================= #
    N = pos.shape[0] # Number of particles
    h = star.h # smoothing length
    dx, dy, dz = getPairwiseSeparations( pos, pos ) # Get pairwise distances
    dWx, dWy, dWz = gradW( dx, dy, dz, h ,kernel_type_)# Get pairwise gradients
    ax = - np.sum( m * ( np.transpose(P/rho**2) + P.T/rho.T**2 ) * dWx, 1).reshape((N,1)) # Add x Pressure contribution to accelerations
    ay = - np.sum( m * ( np.transpose(P/rho**2) + P.T/rho.T**2 ) * dWy, 1).reshape((N,1)) # Add y Pressure contribution to accelerations
    az = - np.sum( m * ( np.transpose(P/rho**2) + P.T/rho.T**2 ) * dWz, 1).reshape((N,1)) # Add z Pressure contribution to accelerations
    a = np.hstack((ax,ay,az)) # pack together the acceleration components
    # ====================== 3. Self-gravity =========================================== #
    r_x,r_y,r_z = self_gravity_radius(dx, dy, dz)
    r = np.sqrt(r_x**2+r_y**2+r_z**2)
    G = star.G_test # gravitational constant (km3 kg-1 s-2). We add a factor of (1 solar mass) in kg becuase star.m (the point mass) is in solar masses.
    Gmr2 = G*star.m/r**2 # G m /r^2.
    ax_grav = Gmr2*r_x
    ay_grav = Gmr2*r_y
    az_grav = Gmr2*r_z
    a[:,0] -= ax_grav
    a[:,1] -= ay_grav
    a[:,2] -= az_grav
    ########====================================########
    
    
    # ====================== 4. Extra forces =========================================== #
    lmbda = clmbda*eos.lmbda(star.mass, star.radius) # external potential force
    a -= lmbda * pos # Add external potential force
    return a

## **__NS Class__**

In [25]:
class NS:
    def __init__(self, tag, eos, mass = 3, radius = 15, num_points = 100,nu=1,lagrange_tag = "vanilla",kernel_type="gaussian",eta=1,clmbda=1):
        """ 
        NEUTRON STAR OBJECT ATTRIBUTES
        tag, mass, radius, m (point mass), num_points, eos, lagrange_tag, kernel_type. eta, nu
        Point attributes: position (N x 3), vel (N x 3), acc (N x 3), density (N x 1), pressure (N x 1).        
        
        Initializes a NS object.
        """
        self.tag = tag # "Name" of the star, just a string for identification
        self.G_test = 1
        self.radius = radius # star radius in km ======= Star properties
        self.mass = mass # star mass in solar Masses 
        self.num_points = num_points # number of SPH points modeling the star
        self.m = mass/num_points # each point has "mass" m in solar masses (maybe change later to baryon number)
        
        self.eos = eos  #eos ======= Equation of state 
        
        self.com = [0,0,0] # center of mass is initialized at the origin. ======= Star motion
        self.f_COM = [0,0,0]  # at initialization there is no net force acting on the center of mass

        np.random.seed(42) # set the random number generator seed ============= Initial pos and veloc ===
        self.pos = np.random.randn(self.num_points,3)   # (km) positions of the particles relative to the center of mass (in km)         # randomly select positions and velocities from initialized seed
        self.vel = np.zeros(self.pos.shape)    # (km/s) randomly select positions and velocities positions and velocities are N x 3 matrices.
        self.acc = np.zeros(self.pos.shape)  # (km/s^2) initially we haven't updated the accelerations of those random points for the star.
        self.points_position = np.copy(self.pos) # true position, this will be corrected when we move the center of mass. 
        
        self.kernel_type = kernel_type # Kernel used for determining particle density ====== Smoothing Length and Particle Density  ==  
        self.h= 0.05 # constant initial smoothing length for initialization
        self.eta = eta # Multiplies the smoothing length based on particle density
        self.rho = getDensity(self.pos, self.pos, self.m, self.h)
        self.h = self.eta*(self.rho/self.m)**(1/3) # Smoothing length depends on density of the particle.
        
        self.ltag = lagrange_tag # ===== Lagrange discrete timestep update equation
        self.l = discrete_lagrange(lagrange_tag)
        
        self.clmbda = clmbda # ===== Potential and artificial viscosity
        self.nu = nu # viscosity
        
        #========= INITIALIZATION STEP. Initialize star from Random points =====#
        dt = 0.05 # timestep for initialization step
        tEnd = 10 # total number of timesteps for initialization
        t = 0 # current time of the simulation
        Nt = int(np.ceil(tEnd/dt)) # number of timesteps
        for i in range(Nt): # Initialization Main Loop
            self.vel += self.acc * dt/2 # (1/2) kick # adds acceleration
            self.pos += self.vel * dt # particle motion
            self.acc = getAcc(self) # get new Nx3 matrix of accelerations based on new pos,v
            self.vel += self.acc * dt/2 # (1/2) kick 
            t += dt # update time
            self.rho = getDensity(self.pos, self.pos, self.m, self.h) # get updated density for each point based on m, h. 
        print("-------success!\n", "\nSTAR OBJECT. Name: '" + self.tag + "', Mass: " + str(self.mass) + " SMs, "+ "Radius: " + str(self.radius) + " km", "\n")
        return None

    def getPosition(self):
        """ 
        Returns the true positions of all the points 
        in the star, which are corrcted for the center of mass location.
        """
        position = np.copy(self.pos)
        COM = self.com
        position[:,0] += COM[0]
        position[:,1] += COM[1]
        position[:,2] += COM[2]
        self.points_position  = position # track the true particle locations
        return position
    
    def move_com(self, com_new): 
        """
        Takes in new coordinates of the center of mass of the star in km. 
        Changes the location of the star's center of mass.
        """
        if len(com_new) != 3:
            raise ValueError("The New Center of Mass you entered for NS is not 3 dimensions.")
        self.com = com_new
        
    def update(self, dt,tag=False): 
        """
        Takes in a timestep. Updates the all the star's SPH points 
        relative to each other for the timestep (dt) based on the 
        current star's eos and SPH point densities. 
        """
        self.rho = getDensity(self.pos, self.pos, self.m, self.h) # Update the density of each point based on the current point positions
        self.h = self.eta*(self.rho/self.m)**(1/3) # Update smoothing length of each particle
        rho = self.rho
        eos = self.eos
        P = eos.P(rho) # Pressure from density using equation of state. 
        L = self.l # discrete lagrangian formulation object
        dVdt = L.dvdt # dvdt from the discrete lagrangian formulation
        dx, dy, dz = getPairwiseSeparations( self.pos, self.pos ) # Get pairwise distances
        dWx, dWy, dWz = gradW( dx, dy, dz, self.h ,self.kernel_type)# Get pairwise gradients
        dvdtx,dvdty,dvdtz = dVdt(self.m, self.num_points, P, rho, self.vel, dWx,dWy,dWz,self.nu)  # Lagrange inputs: m, M, dW,h,P, rho,dx,dy,dz
        dvdtx = np.transpose(dvdtx)
        dvdty = np.transpose(dvdty)
        dvdtz = np.transpose(dvdtz)
        vx = np.copy(np.asarray([list(self.vel[:,0])]))
        vy = np.copy(np.asarray([list(self.vel[:,1])]))
        vz = np.copy(np.asarray([list(self.vel[:,2])]))
        vx = vx + dvdtx*dt # Add x velocity change for this timestep
        vy = vy + dvdty*dt # Add y velocity change for this timestep
        vz = vz + dvdtz*dt # Add z velocity change for this timestep
        v = np.hstack((np.transpose(vx),np.transpose(vy),np.transpose(vz))) # pack together the acceleration components
        self.vel = 1e-20*v # update velocity
        pos1= self.pos
        self.pos = self.pos + dt*v*1e-20 # update position
        return self.pos # returns the point's new positions at each update step
    
    def __str__(self):
        return "\nSTAR OBJECT. Name: '" + self.tag + "', Mass: " + str(self.mass) + " SMs, "+ "Radius: " + str(self.radius) + " km" 
    def get_relative_position(self):
        """  This function returns an N x 3 array of the star's points positions relative to the center of mass """
        return self.pos
    def get_velocity(self):
        """ This function returns the star's velocity.    """
        return self.vel
    def get_mass(self):
        """ This function returns the star's overall mass . """
        return self.mass

_______________________________________________________________
# Initialization Testng (below)

## Test star initialization and update

In [24]:
#++++++++++++
Mass = 10
Radius = 2
points = 50
#++++++++++++
eos = EOS('adiabatic');
star_1 = NS("star_1", eos,Mass,Radius,num_points = points,kernel_type = "wendland_2");# Initialize star
print("1 Initialized star")
star_1.update(0.1); # supress output
print("2 Update ran")

-------success!
 
STAR OBJECT. Name: 'star_1', Mass: 10 SMs, Radius: 2 km 

1 Initialized star
2 Update ran


## **__Info File Function__**
Produces a string containg information about the star to put in a file.

In [ ]:
def get_file_info(date,ns,eos, dt,tEnd,notes = "None"):
    file_info = 'Date: '+ date
    file_info += "\n\n Number of points: "+str(ns.num_points)
    file_info += "\n dt: " + str(dt) + " seconds"
    file_info += "\n total simulation time: " + str(tEnd) + " seconds"
    file_info += "\n Mass: " + str(ns.mass) + " SMs"
    file_info += "\n Radius: " + str(ns.radius) + " km"    
    file_info += "\n h: "+ str(ns.h)
    file_info += "\n v: "+ str(ns.nu)
    file_info += "\n nu: " + str(ns.nu)
    file_info += "\nLagrange_tag: " + ns.ltag
    file_info += "\nEOS: : " + eos.name
    file_info += "\nKernel: : " + ns.kernel_type
    file_info += "\n\nnotes: " + notes
    return file_info 

## **__ANIMATION CODE__**

In [ ]:
# def update_graph(num):
#     data=df[df['time']==num]
#     graph._offsets3d = (data.x, data.y, data.z)
#     title.set_text('3D Test, time={}'.format(num)) # Mass = 10
# Radius = 2
# points = 50

# eos = EOS('standard') 
# star_1 = NS("star_1", eos,Mass,Radius,num_points = points,kernel_type = "wendland_2") # Initialize star
# star_1.update(0.1); # supress output
# a = np.asarray(star_1.pos) # Points for plotting
# points_count = len(a) # Total number of points
# points_per_timestep = star_1.num_points # assign all points to the same timestep in the movie
# t = np.array([np.ones(points_per_timestep)*i for i in range(int(points_count/points_per_timestep))]).flatten() 
# df = pd.DataFrame({"time": t ,"x" : a[:,0], "y" : a[:,1], "z" : a[:,2]})
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# title = ax.set_title('NS Test')
# data=df[df['time']==0]
# graph = ax.scatter(data.x, data.y, data.z)
# ani = matplotlib.animation.FuncAnimation(fig, update_graph, 1, interval=10, blit=False)
# plt.show()
# ani.save("eos_test" + str(Mass) + "_" + str(Radius) +"_" +  str(points) + ".mp4")

In [ ]:
#     def modify_com_force(self, f_COM_new): 
#         """ Modifies the force acting on the center of mass of the star. """
#         if len(f_COM_new) != 3:
#             raise ValueError("The force acting on the COM that you entered for the NS is not in 3 dimensions.")
#         self.f_COM = f_COM_new
    